<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/HVAC_%EC%B5%9C%EC%A0%81%ED%99%94_%EC%8B%9C%EC%8A%A4%ED%85%9C_Python_%EC%A0%84%EC%B2%B4_%EC%BD%94%EB%93%9C_(%EC%83%81%EC%84%B8_%EB%8D%94%EB%AF%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B0%8F_%EB%AA%A8%EB%8D%B8_%ED%95%99%EC%8A%B5_%ED%8F%AC%ED%95%A8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# import joblib # 실제 모델 저장/로드를 위해 사용 가능

class EnthalpyCalculator:
    def calculate_enthalpy(self, temperature_c, relative_humidity_percent, pressure_pa=101325):
        # 매우 단순화된 placeholder 유지
        return (temperature_c * 1000 + (relative_humidity_percent / 100) * 2500 * 10) * (1 + (temperature_c / 100))

    def calculate_temp_humidity_from_enthalpy(self, enthalpy_j_kg, initial_temp_c, pressure_pa=101325):
        initial_h_approx = self.calculate_enthalpy(initial_temp_c, 50)
        delta_h = enthalpy_j_kg - initial_h_approx
        # 비열 근사치 (J/kg.K) - 건공기 + 수증기 고려 (습도에 따라 변함)
        # cp_air = 1005, cp_vapor = 1860. x = 절대습도 (kg_vapor/kg_dry_air)
        # cp_humid_air = cp_air + x * cp_vapor
        # 단순화를 위해 평균적인 절대습도(0.01 kg/kg)를 가정하여 cp_humid_air ~ 1005 + 0.01 * 1860 ~ 1023.6
        cp_approx = 1025
        delta_temp_c = delta_h / cp_approx

        new_temp_c = initial_temp_c + delta_temp_c # 엔탈피 변화가 모두 현열 변화로 간다고 가정 (단순화)

        # 습도 변화 로직은 매우 복잡하며, 엔탈피와 온도만으로는 정확한 계산이 어려움.
        # 여기서는 온도가 변하면 포화수증기압이 변하고, 이에 따라 상대습도가 변한다고 가정. (매우 단순화)
        # 또는, 잠열 변화가 없다고 가정하면 절대습도 일정 -> 상대습도 변화.
        # 여기서는 이전의 임의적 로직을 유지하되, 변화폭을 줄임.
        humidity_change_factor = (enthalpy_j_kg - initial_h_approx) / 20000.0 # 변화폭 줄임
        new_humidity_percent = initial_temp_c / new_temp_c * 50 if new_temp_c !=0 else 50 # 온도가 변하면 RH도 변한다고 가정
        new_humidity_percent = 50 + humidity_change_factor * 5


        new_humidity_percent = min(max(new_humidity_percent, 5.0), 99.0)
        new_temp_c = min(max(new_temp_c, -15.0), 55.0) # 온도 범위 현실화
        return new_temp_c, new_humidity_percent

# --- 상세 더미 데이터 생성 함수 ---
def generate_dummy_data(num_rows=1000, num_oac=14, num_low_ch=8, num_high_ch=8, num_ct=9):
    """학습 및 테스트를 위한 상세 더미 시계열 데이터를 생성합니다."""
    print(f"상세 더미 데이터 생성 시작: {num_rows} 행, OAC {num_oac}대, 저온냉동기 {num_low_ch}대, 고온냉동기 {num_high_ch}대, 냉각탑 {num_ct}대")
    data_dict = {} # DataFrame 생성을 위한 딕셔너리

    datetimes = pd.to_datetime('2023-01-01') + pd.to_timedelta(np.arange(num_rows) * 1, unit='h')
    data_dict['datetime'] = datetimes

    # 외기 조건
    outdoor_temps = 15 + 10 * np.sin(2 * np.pi * np.arange(num_rows) / (24 * 30)) + 5 * np.sin(2 * np.pi * np.arange(num_rows) / 24) + np.random.normal(0, 1, num_rows)
    outdoor_rhs = 60 + 15 * np.sin(2 * np.pi * np.arange(num_rows) / (24*15)) + 10 * np.sin(2*np.pi*np.arange(num_rows)/24 + np.pi/2) + np.random.normal(0,5,num_rows)
    outdoor_rhs = np.clip(outdoor_rhs, 20, 95)
    data_dict['outdoor_temp_c'] = outdoor_temps
    data_dict['outdoor_rh_percent'] = outdoor_rhs

    enthalpy_calc = EnthalpyCalculator()
    oac_coil_types = ['preheating', 'precooling', 'cooling', 'reheating', 'humidification']
    design_delta_h_oac = {'preheating': 15000, 'precooling': -20000, 'cooling': -25000, 'reheating': 12000, 'humidification': 8000}

    total_precooling_load_watts_series = np.zeros(num_rows)
    total_cooling_load_watts_series = np.zeros(num_rows)

    # OAC 데이터 (14대)
    for i in range(num_oac):
        current_oac_temp_c = outdoor_temps.copy()
        current_oac_rh_percent = outdoor_rhs.copy()

        for row_idx in range(num_rows): # 각 시간 스텝별로 OAC 내부 공기 상태 변화 시뮬레이션
            oac_in_temp = outdoor_temps[row_idx]
            oac_in_rh = outdoor_rhs[row_idx]

            for coil_idx, coil_name in enumerate(oac_coil_types):
                # 각 OAC, 각 코일, 각 시간스텝별 데이터 생성
                # set_point 생성 (약간의 변동성 추가)
                sp_temp_key = f'oac_{i}_{coil_name}_set_point_temp_c'
                sp_rate_key = f'oac_{i}_{coil_name}_set_point_rate' # 가습용
                open_rate_key = f'oac_{i}_{coil_name}_coil_open_rate'
                outlet_temp_key = f'oac_{i}_{coil_name}_후단온도_c'
                outlet_rh_key = f'oac_{i}_{coil_name}_후단습도_percent'

                if row_idx == 0: # 첫 행일 때만 컬럼 초기화
                    if coil_name == 'humidification':
                        data_dict[sp_rate_key] = np.zeros(num_rows)
                    else:
                        data_dict[sp_temp_key] = np.zeros(num_rows)
                    data_dict[open_rate_key] = np.zeros(num_rows)
                    data_dict[outlet_temp_key] = np.zeros(num_rows)
                    data_dict[outlet_rh_key] = np.zeros(num_rows)

                # Set point 설정 (시간별, OAC별 변동 가능)
                set_point_val = 0
                if coil_name == 'preheating': set_point_val = np.where(oac_in_temp < 18, 20, oac_in_temp) + np.random.normal(0,0.2)
                elif coil_name == 'precooling': set_point_val = np.where(oac_in_temp > 25, 22, oac_in_temp) + np.random.normal(0,0.2)
                elif coil_name == 'cooling': set_point_val = 12.0 + np.random.normal(0,0.2)
                elif coil_name == 'reheating': set_point_val = 22.0 + np.random.normal(0,0.2)
                elif coil_name == 'humidification': set_point_val = np.random.uniform(0,0.3) if outdoor_temps[row_idx] < 15 else 0.0 # 겨울철 주로 가습

                if coil_name == 'humidification':
                    data_dict[sp_rate_key][row_idx] = set_point_val
                else:
                    data_dict[sp_temp_key][row_idx] = set_point_val

                # 개도율 계산 (규칙 기반 단순화)
                open_rate = 0.0
                inlet_h = enthalpy_calc.calculate_enthalpy(oac_in_temp, oac_in_rh)
                if coil_name == 'humidification':
                    open_rate = set_point_val * np.random.uniform(0.8, 1.2)
                else:
                    target_h = enthalpy_calc.calculate_enthalpy(set_point_val, oac_in_rh) # 목표습도 현재와 동일가정
                    required_delta_h = target_h - inlet_h
                    if design_delta_h_oac[coil_name] != 0:
                        if (design_delta_h_oac[coil_name] > 0 and required_delta_h > 0) or \
                           (design_delta_h_oac[coil_name] < 0 and required_delta_h < 0):
                            open_rate = abs(required_delta_h / design_delta_h_oac[coil_name])
                open_rate = np.clip(open_rate, 0, 1) * np.random.uniform(0.9, 1.1) # 약간의 노이즈
                open_rate = np.clip(open_rate, 0, 1)
                data_dict[open_rate_key][row_idx] = open_rate

                # 출구 상태 계산
                actual_delta_h = design_delta_h_oac[coil_name] * open_rate
                outlet_h = inlet_h + actual_delta_h
                oac_out_temp, oac_out_rh = enthalpy_calc.calculate_temp_humidity_from_enthalpy(outlet_h, oac_in_temp)

                data_dict[outlet_temp_key][row_idx] = oac_out_temp
                data_dict[outlet_rh_key][row_idx] = oac_out_rh

                # 다음 코일의 입구는 현재 코일의 출구
                oac_in_temp = oac_out_temp
                oac_in_rh = oac_out_rh

                # 시스템 전체 부하 누적 (12kg/s 공기유량 가정)
                air_mass_flow_per_oac = 12.0
                if coil_name == 'precooling':
                    total_precooling_load_watts_series[row_idx] += abs(actual_delta_h * air_mass_flow_per_oac)
                elif coil_name == 'cooling':
                    total_cooling_load_watts_series[row_idx] += abs(actual_delta_h * air_mass_flow_per_oac)

            # OAC 최종 토출 상태
            if row_idx == 0:
                data_dict[f'oac_{i}_토출온도_c'] = np.zeros(num_rows)
                data_dict[f'oac_{i}_토출노점온도_c'] = np.zeros(num_rows) # 노점온도 계산은 복잡하므로 여기서는 임의값 또는 생략
                data_dict[f'oac_{i}_토출압력_pa'] = np.zeros(num_rows)   # 토출압력은 일정하다고 가정 또는 임의값

            data_dict[f'oac_{i}_토출온도_c'][row_idx] = oac_out_temp
            data_dict[f'oac_{i}_토출노점온도_c'][row_idx] = oac_out_temp - 2 # 임의의 노점 (실제 계산 필요)
            data_dict[f'oac_{i}_토출압력_pa'][row_idx] = 101325 + np.random.uniform(100,300) # 약간의 양압 가정

    data_dict['total_precooling_load_watts'] = total_precooling_load_watts_series
    data_dict['total_cooling_load_watts'] = total_cooling_load_watts_series

    # 저온/고온 냉동기 데이터 (각 8대)
    for temp_type_prefix, num_chillers in [("low_temp_chiller", num_low_ch), ("high_temp_chiller", num_high_ch)]:
        load_key = 'total_cooling_load_watts' if "low_temp" in temp_type_prefix else 'total_precooling_load_watts'

        # 메인 시스템 값
        data_dict[f'{temp_type_prefix}_main_차압압력_bar'] = 1.5 + np.random.normal(0, 0.1, num_rows) - (data_dict[load_key]/np.mean(data_dict[load_key] + 1e-6) -1)*0.2
        data_dict[f'{temp_type_prefix}_main_차압개도율_percent'] = 60 + np.random.normal(0, 5, num_rows) + (data_dict[load_key]/np.mean(data_dict[load_key] + 1e-6) -1)*10
        data_dict[f'{temp_type_prefix}_sub_차압개도율_percent'] = 50 + np.random.normal(0, 5, num_rows) + (data_dict[load_key]/np.mean(data_dict[load_key] + 1e-6) -1)*8
        data_dict[f'{temp_type_prefix}_main_supply_압력_pa'] = 200000 + np.random.uniform(-5000, 5000, num_rows)
        data_dict[f'{temp_type_prefix}_main_supply_온도_c'] = (7.0 if "low_temp" in temp_type_prefix else 10.0) + np.random.normal(0, 0.2, num_rows)
        data_dict[f'{temp_type_prefix}_main_return_압력_pa'] = 150000 + np.random.uniform(-5000, 5000, num_rows)
        data_dict[f'{temp_type_prefix}_main_return_온도_c'] = data_dict[f'{temp_type_prefix}_main_supply_온도_c'] + (data_dict[load_key] / (num_chillers * 150000 + 1e-6)) * 5 # 부하에 따라 리턴온도 상승폭 변화 (단순화)

        data_dict[f'{temp_type_prefix}_main_차압압력_bar'] = np.clip(data_dict[f'{temp_type_prefix}_main_차압압력_bar'], 1.0, 2.5)
        data_dict[f'{temp_type_prefix}_main_차압개도율_percent'] = np.clip(data_dict[f'{temp_type_prefix}_main_차압개도율_percent'], 10, 100)
        data_dict[f'{temp_type_prefix}_sub_차압개도율_percent'] = np.clip(data_dict[f'{temp_type_prefix}_sub_차압개도율_percent'], 10, 100)
        data_dict[f'{temp_type_prefix}_main_return_온도_c'] = np.clip(data_dict[f'{temp_type_prefix}_main_return_온도_c'],
                                                                data_dict[f'{temp_type_prefix}_main_supply_온도_c']+0.5 ,
                                                                data_dict[f'{temp_type_prefix}_main_supply_온도_c']+10)


        # 개별 냉동기 값
        active_count_series = np.ceil(data_dict[load_key] / (200000 if "low_temp" in temp_type_prefix else 250000)).astype(int) # 대당 용량 가정
        active_count_series = np.clip(active_count_series, 1, num_chillers)
        data_dict[f'{temp_type_prefix}_active_count'] = active_count_series # ChillerModel 학습용

        for j in range(num_chillers):
            # j번째 냉동기가 가동 중일 확률 (active_count에 따라)
            is_active_prob = active_count_series / num_chillers
            is_active_roll = np.random.rand(num_rows) < is_active_prob

            base_supply_temp = data_dict[f'{temp_type_prefix}_main_supply_온도_c']
            data_dict[f'{temp_type_prefix}_{j}_supply_냉수온도_c'] = np.where(is_active_roll, base_supply_temp + np.random.normal(0,0.1,num_rows), base_supply_temp + np.random.uniform(0.5,2,num_rows) ) # 미가동시 온도 약간 높음
            data_dict[f'{temp_type_prefix}_{j}_return_냉수온도_c'] = data_dict[f'{temp_type_prefix}_{j}_supply_냉수온도_c'] + \
                                                                np.where(is_active_roll, (data_dict[load_key]/(active_count_series+1e-6))/(150000+1e-6) * 5 + np.random.normal(0,0.2,num_rows) , np.random.uniform(0.1,0.5,num_rows))
            data_dict[f'{temp_type_prefix}_{j}_supply_냉각수온도_c'] = outdoor_temps + 5 + np.random.normal(0,0.5,num_rows) # 냉각수 온도는 외기 + alpha
            data_dict[f'{temp_type_prefix}_{j}_return_냉각수온도_c'] = data_dict[f'{temp_type_prefix}_{j}_supply_냉각수온도_c'] + \
                                                                    np.where(is_active_roll, 3 + np.random.normal(0,0.3,num_rows), np.random.uniform(0.1,0.5,num_rows))

            data_dict[f'{temp_type_prefix}_{j}_return_냉수온도_c'] = np.clip(data_dict[f'{temp_type_prefix}_{j}_return_냉수온도_c'],
                                                                        data_dict[f'{temp_type_prefix}_{j}_supply_냉수온도_c']+0.2,
                                                                        data_dict[f'{temp_type_prefix}_{j}_supply_냉수온도_c']+12)
            data_dict[f'{temp_type_prefix}_{j}_return_냉각수온도_c'] = np.clip(data_dict[f'{temp_type_prefix}_{j}_return_냉각수온도_c'],
                                                                        data_dict[f'{temp_type_prefix}_{j}_supply_냉각수온도_c']+0.2,
                                                                        data_dict[f'{temp_type_prefix}_{j}_supply_냉각수온도_c']+10)


    # 냉각탑 데이터 (9대)
    for k in range(num_ct):
        data_dict[f'cooling_tower_{k}_supply_냉각수온도_c'] = outdoor_temps + 3 + np.random.normal(0,0.5,num_rows) # 외기온도 + 접근온도차
        data_dict[f'cooling_tower_{k}_return_냉각수온도_c'] = data_dict[f'cooling_tower_{k}_supply_냉각수온도_c'] + np.random.uniform(3,7,num_rows) # 냉각 전후 온도차
        data_dict[f'cooling_tower_{k}_supply_수조레벨_percent'] = np.random.uniform(70,95,num_rows)

        data_dict[f'cooling_tower_{k}_return_냉각수온도_c'] = np.clip(data_dict[f'cooling_tower_{k}_return_냉각수온도_c'],
                                                                    data_dict[f'cooling_tower_{k}_supply_냉각수온도_c'] + 1.0,
                                                                    data_dict[f'cooling_tower_{k}_supply_냉각수온도_c'] + 15)


    df = pd.DataFrame(data_dict)
    print(f"상세 더미 데이터 생성 완료. DataFrame Shape: {df.shape}")
    return df


class OACModel:
    def __init__(self, num_oac_units=14): # 이 num_oac_units는 Facade에서 총 OAC 대수를 의미. 모델 자체는 대표 OAC 하나를 학습.
        self.num_oac_units = num_oac_units
        self.enthalpy_calculator = EnthalpyCalculator()
        self.coil_order = ['preheating', 'precooling', 'cooling', 'reheating', 'humidification']

        self.trained_models = {}
        self.design_delta_h = {
            'preheating': 15000, 'precooling': -20000, 'cooling': -25000,
            'reheating': 12000, 'humidification': 8000
        }
        self.feature_names_per_coil = {} # 학습 시 사용된 피처 이름 저장

    def train_coil_models(self, df_train_data_full):
        print("\n--- OAC 코일 모델 학습 시작 (대표 OAC: oac_0 사용) ---")

        df_train_data_full['outdoor_enthalpy'] = self.enthalpy_calculator.calculate_enthalpy(
            df_train_data_full['outdoor_temp_c'], df_train_data_full['outdoor_rh_percent']
        )

        # 대표 OAC (oac_0)의 데이터만 사용하여 모델 학습
        # 컬럼 이름에서 'oac_0_' 부분을 일반적인 이름으로 매핑하여 사용
        representative_oac_idx = 0 # 0번 OAC를 대표로 사용

        for coil_type in self.coil_order:
            print(f"  {coil_type} 코일 모델 학습 중...")

            target_col = f'oac_{representative_oac_idx}_{coil_type}_coil_open_rate'
            if target_col not in df_train_data_full.columns:
                print(f"    경고: 대표 OAC 타겟 컬럼 {target_col}이 없어 {coil_type} 모델 학습을 건너<0xEB><0><0x84>니다.")
                continue

            temp_df_train = df_train_data_full.copy()
            current_features_generic_names = [] # 일반화된 피처 이름 (oac_0_ 접두사 없이)
            current_features_specific_names = [] # 실제 데이터프레임 컬럼 이름 (oac_0_ 접두사 포함)


            # 기본 외기 조건
            current_features_generic_names.extend(['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy'])
            current_features_specific_names.extend(['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy'])

            if coil_type == 'preheating':
                current_features_generic_names.append('current_coil_set_point_temp_c')
                current_features_specific_names.append(f'oac_{representative_oac_idx}_{coil_type}_set_point_temp_c')
            elif coil_type == 'humidification':
                prev_coil = self.coil_order[self.coil_order.index(coil_type)-1]
                current_features_generic_names.extend([f'prev_coil_outlet_temp_c', f'prev_coil_outlet_rh_percent', f'prev_coil_outlet_h_actual'])
                current_features_specific_names.extend([f'oac_{representative_oac_idx}_{prev_coil}_후단온도_c',
                                                        f'oac_{representative_oac_idx}_{prev_coil}_후단습도_percent',
                                                        f'oac_{representative_oac_idx}_{prev_coil}_후단엔탈피_calculated']) # 계산된 엔탈피 컬럼
                temp_df_train[f'oac_{representative_oac_idx}_{prev_coil}_후단엔탈피_calculated'] = self.enthalpy_calculator.calculate_enthalpy(
                    temp_df_train[f'oac_{representative_oac_idx}_{prev_coil}_후단온도_c'], temp_df_train[f'oac_{representative_oac_idx}_{prev_coil}_후단습도_percent']
                )
                current_features_generic_names.append('current_coil_set_point_rate')
                current_features_specific_names.append(f'oac_{representative_oac_idx}_{coil_type}_set_point_rate')
            else: # precooling, cooling, reheating
                prev_coil = self.coil_order[self.coil_order.index(coil_type)-1]
                current_features_generic_names.extend([f'prev_coil_outlet_temp_c', f'prev_coil_outlet_rh_percent', f'prev_coil_outlet_h_actual'])
                current_features_specific_names.extend([f'oac_{representative_oac_idx}_{prev_coil}_후단온도_c',
                                                        f'oac_{representative_oac_idx}_{prev_coil}_후단습도_percent',
                                                        f'oac_{representative_oac_idx}_{prev_coil}_후단엔탈피_calculated'])
                temp_df_train[f'oac_{representative_oac_idx}_{prev_coil}_후단엔탈피_calculated'] = self.enthalpy_calculator.calculate_enthalpy(
                    temp_df_train[f'oac_{representative_oac_idx}_{prev_coil}_후단온도_c'], temp_df_train[f'oac_{representative_oac_idx}_{prev_coil}_후단습도_percent']
                )
                current_features_generic_names.append('current_coil_set_point_temp_c')
                current_features_specific_names.append(f'oac_{representative_oac_idx}_{coil_type}_set_point_temp_c')

            self.feature_names_per_coil[coil_type] = current_features_generic_names # 예측 시 사용할 일반화된 피처 이름 저장

            X = temp_df_train[current_features_specific_names].fillna(method='ffill').fillna(method='bfill')
            X.columns = current_features_generic_names # 컬럼명을 일반화된 이름으로 변경하여 모델 학습
            y = temp_df_train[target_col]

            if X.empty or y.empty or X.shape[0] != y.shape[0]:
                print(f"    오류: 특징 또는 타겟 데이터가 비어있거나 크기가 맞지 않아 {coil_type} 모델 학습을 건너<0xEB><0><0x84>니다.")
                continue

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1, max_depth=10, min_samples_split=10, min_samples_leaf=5)
            model.fit(X_train, y_train)

            y_pred_test = model.predict(X_test)
            mse_test = mean_squared_error(y_test, y_pred_test)
            print(f"    {coil_type} 모델 학습 완료. Test MSE: {mse_test:.4f}")
            self.trained_models[coil_type] = model

        print("--- OAC 코일 모델 학습 완료 ---")


    def predict_coil_open_rates(self, outdoor_temp_c, outdoor_rh_percent, oac_set_points, modified_coil_states=None):
        if not self.trained_models or not self.feature_names_per_coil : # 모델 또는 피처 이름 정보가 없으면 규칙 기반
            # print("경고: OAC 모델 또는 피처 정보가 학습되지 않았습니다. 규칙 기반으로 예측합니다.")
            return self._predict_coil_open_rates_rule_based(outdoor_temp_c, outdoor_rh_percent, oac_set_points, modified_coil_states)

        if modified_coil_states is None: modified_coil_states = {}

        current_temp_c_for_sim = outdoor_temp_c # OAC 내부 공기 상태 시뮬레이션용
        current_rh_percent_for_sim = outdoor_rh_percent

        outdoor_enthalpy_val = self.enthalpy_calculator.calculate_enthalpy(outdoor_temp_c, outdoor_rh_percent)
        results = {}

        prev_coil_actual_outlet_temp_for_pred = outdoor_temp_c # 다음 코일 예측 시 사용될 이전 코일 출구 (실제값 기준)
        prev_coil_actual_outlet_rh_for_pred = outdoor_rh_percent
        # prev_coil_actual_outlet_h_for_pred # 루프 내에서 계산

        for coil_type in self.coil_order:
            inlet_temp_c_sim = current_temp_c_for_sim # 현재 코일 입구 (시뮬레이션)
            inlet_rh_percent_sim = current_rh_percent_for_sim
            inlet_h_j_kg_sim = self.enthalpy_calculator.calculate_enthalpy(inlet_temp_c_sim, inlet_rh_percent_sim)

            predicted_open_rate = 0.0

            if coil_type in modified_coil_states:
                predicted_open_rate = modified_coil_states[coil_type]
            elif coil_type in self.trained_models:
                model = self.trained_models[coil_type]
                generic_feature_names = self.feature_names_per_coil.get(coil_type)
                if not generic_feature_names:
                    print(f"    경고: {coil_type} 모델의 피처 이름 정보가 없습니다. 개도율 0으로 설정.")
                    predicted_open_rate = 0.0
                else:
                    feature_values_map = {
                        'outdoor_temp_c': outdoor_temp_c,
                        'outdoor_rh_percent': outdoor_rh_percent,
                        'outdoor_enthalpy': outdoor_enthalpy_val,
                        'prev_coil_outlet_temp_c': prev_coil_actual_outlet_temp_for_pred,
                        'prev_coil_outlet_rh_percent': prev_coil_actual_outlet_rh_for_pred,
                        'prev_coil_outlet_h_actual': self.enthalpy_calculator.calculate_enthalpy(prev_coil_actual_outlet_temp_for_pred, prev_coil_actual_outlet_rh_for_pred),
                        'current_coil_set_point_temp_c': oac_set_points.get(coil_type, inlet_temp_c_sim), # SP 없으면 현재 온도로
                        'current_coil_set_point_rate': oac_set_points.get('humidification_rate', 0.0) # 가습용
                    }

                    feature_values_for_pred_list = [feature_values_map[name] for name in generic_feature_names]

                    try:
                        input_df = pd.DataFrame([feature_values_for_pred_list], columns=generic_feature_names)
                        predicted_open_rate = model.predict(input_df)[0]
                        predicted_open_rate = np.clip(predicted_open_rate, 0.0, 1.0)
                    except Exception as e:
                        print(f"    오류: {coil_type} 모델 예측 중 오류 발생 - {e}. 개도율 0으로 설정.")
                        predicted_open_rate = 0.0
            else:
                 predicted_open_rate = 0.0

            actual_delta_h_j_kg = self.design_delta_h[coil_type] * predicted_open_rate
            outlet_h_j_kg_sim = inlet_h_j_kg_sim + actual_delta_h_j_kg
            current_temp_c_for_sim, current_rh_percent_for_sim = self.enthalpy_calculator.calculate_temp_humidity_from_enthalpy(outlet_h_j_kg_sim, inlet_temp_c_sim)

            results[coil_type] = {
                'open_rate': predicted_open_rate,
                'inlet_temp_c': inlet_temp_c_sim, 'inlet_rh_percent': inlet_rh_percent_sim, 'inlet_h_j_kg': inlet_h_j_kg_sim,
                'outlet_temp_c': current_temp_c_for_sim, 'outlet_rh_percent': current_rh_percent_for_sim, 'outlet_h_j_kg': outlet_h_j_kg_sim,
                'delta_h_j_kg': actual_delta_h_j_kg,
                'target_outlet_temp_c': oac_set_points.get(coil_type, 'N/A') if coil_type != 'humidification' else oac_set_points.get('humidification_rate','N/A')
            }

            prev_coil_actual_outlet_temp_for_pred = current_temp_c_for_sim # 다음 코일 예측을 위해 현재 코일의 시뮬레이션 출구값 사용
            prev_coil_actual_outlet_rh_for_pred = current_rh_percent_for_sim

        results['final_outlet_temp_c'] = current_temp_c_for_sim
        results['final_outlet_rh_percent'] = current_rh_percent_for_sim
        results['final_outlet_h_j_kg'] = self.enthalpy_calculator.calculate_enthalpy(current_temp_c_for_sim, current_rh_percent_for_sim)
        return results

    def _predict_coil_open_rates_rule_based(self, outdoor_temp_c, outdoor_rh_percent, oac_set_points, modified_coil_states=None):
        if modified_coil_states is None: modified_coil_states = {}
        current_temp_c = outdoor_temp_c
        current_rh_percent = outdoor_rh_percent
        current_h_j_kg = self.enthalpy_calculator.calculate_enthalpy(current_temp_c, current_rh_percent)
        results = {}
        for coil_type in self.coil_order:
            inlet_temp_c, inlet_rh_percent, inlet_h_j_kg = current_temp_c, current_rh_percent, current_h_j_kg
            target_outlet_temp_c = oac_set_points.get(coil_type)
            predicted_open_rate, actual_delta_h_j_kg = 0.0, 0.0

            if coil_type in modified_coil_states:
                predicted_open_rate = modified_coil_states[coil_type]
            elif coil_type == 'humidification':
                predicted_open_rate = oac_set_points.get('humidification_rate', 0.0)
            elif target_outlet_temp_c is not None:
                target_outlet_h_j_kg = self.enthalpy_calculator.calculate_enthalpy(target_outlet_temp_c, current_rh_percent)
                required_delta_h_j_kg = target_outlet_h_j_kg - inlet_h_j_kg
                if self.design_delta_h[coil_type] > 0:
                    predicted_open_rate = required_delta_h_j_kg / self.design_delta_h[coil_type] if required_delta_h_j_kg > 0 and self.design_delta_h[coil_type] !=0 else 0.0
                elif self.design_delta_h[coil_type] < 0:
                    predicted_open_rate = required_delta_h_j_kg / self.design_delta_h[coil_type] if required_delta_h_j_kg < 0 and self.design_delta_h[coil_type] !=0 else 0.0
                else: predicted_open_rate = 0.0
                predicted_open_rate = np.clip(predicted_open_rate, 0.0, 1.0)

            actual_delta_h_j_kg = self.design_delta_h[coil_type] * predicted_open_rate
            outlet_h_j_kg = inlet_h_j_kg + actual_delta_h_j_kg
            current_temp_c, current_rh_percent = self.enthalpy_calculator.calculate_temp_humidity_from_enthalpy(outlet_h_j_kg, inlet_temp_c)
            current_h_j_kg = outlet_h_j_kg
            results[coil_type] = {
                'open_rate': predicted_open_rate, 'inlet_temp_c': inlet_temp_c, 'inlet_rh_percent': inlet_rh_percent,
                'inlet_h_j_kg': inlet_h_j_kg, 'outlet_temp_c': current_temp_c, 'outlet_rh_percent': current_rh_percent,
                'outlet_h_j_kg': current_h_j_kg, 'delta_h_j_kg': actual_delta_h_j_kg,
                'target_outlet_temp_c': target_outlet_temp_c if coil_type != 'humidification' else oac_set_points.get('humidification_rate','N/A')
            }
        results['final_outlet_temp_c'], results['final_outlet_rh_percent'], results['final_outlet_h_j_kg'] = current_temp_c, current_rh_percent, current_h_j_kg
        return results


class ChillerModel:
    def __init__(self):
        self.trained_models = {}
        self.feature_names_chiller = {} # 타입별 학습 피처 이름 저장

    def train_pressure_valve_models(self, df_train_data):
        print("\n--- 냉동기 차압/밸브 모델 학습 시작 ---")
        for chiller_type in ["low_temp", "high_temp"]: # ChillerModel은 시스템 레벨 예측
            print(f"  {chiller_type}_chiller 시스템 모델 학습 중...")

            # 이 모델은 시스템 전체 부하와 해당 타입 냉동기 총 가동대수를 입력으로 받음
            load_col_for_training = 'total_cooling_load_watts' if chiller_type == "low_temp" else 'total_precooling_load_watts'
            active_count_col_for_training = f'{chiller_type}_chiller_active_count' # 더미 데이터에서 생성된 컬럼명

            current_feature_cols = [load_col_for_training, active_count_col_for_training]
            self.feature_names_chiller[chiller_type] = current_feature_cols # 예측 시 사용할 피처 이름 저장

            # 예측 대상 타겟 컬럼들 (시스템 레벨)
            target_cols_for_training = [f'{chiller_type}_chiller_main_차압압력_bar',
                                        f'{chiller_type}_chiller_main_차압개도율_percent',
                                        f'{chiller_type}_chiller_sub_차압개도율_percent']
                                        # 메인 supply/return 등은 직접 예측 대상이 아님 (최적화 결과 또는 상태값)

            if not all(col in df_train_data.columns for col in current_feature_cols):
                print(f"    경고: 특징 컬럼({current_feature_cols})이 없어 {chiller_type} 모델 학습을 건너<0xEB><0><0x84>니다.")
                continue
            if not all(col in df_train_data.columns for col in target_cols_for_training):
                print(f"    경고: 타겟 컬럼 중 일부가 없어 {chiller_type} 모델 학습을 건너<0xEB><0><0x84>니다.")
                continue

            X = df_train_data[current_feature_cols].fillna(method='ffill').fillna(method='bfill')

            type_models = {}
            for target_col in target_cols_for_training:
                y = df_train_data[target_col]
                if X.empty or y.empty or X.shape[0] != y.shape[0]:
                    print(f"    오류: 특징 또는 타겟({target_col}) 데이터 문제로 학습 건너<0xEB><0><0x84>니다.")
                    continue

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                model = RandomForestRegressor(n_estimators=30, random_state=42, n_jobs=-1, max_depth=8, min_samples_split=8,min_samples_leaf=4)
                model.fit(X_train, y_train)

                y_pred_test = model.predict(X_test)
                mse_test = mean_squared_error(y_test, y_pred_test)
                # 출력 키를 일반적인 이름으로 (예: 'main_차압압력_bar')
                output_key = target_col.replace(f'{chiller_type}_chiller_', '')
                print(f"    {output_key} 모델 학습 완료. Test MSE: {mse_test:.4f}")
                type_models[output_key] = model # 일반화된 키로 모델 저장
            self.trained_models[chiller_type] = type_models
        print("--- 냉동기 차압/밸브 모델 학습 완료 ---")

    def predict_pressure_valve(self, current_total_load_watts, num_active_chillers_of_type, chiller_type="low_temp"):
        if chiller_type not in self.trained_models or not self.trained_models[chiller_type] or \
           chiller_type not in self.feature_names_chiller:
            # print(f"경고: {chiller_type} 냉동기 모델/피처 정보가 학습되지 않았습니다. 기본값을 반환합니다.")
            return {"main_차압압력_bar": 1.5, "main_차압개도율_percent": 60.0, "sub_차압개도율_percent": 50.0} # 키 이름 수정

        models_for_type = self.trained_models[chiller_type]
        feature_cols_for_prediction = self.feature_names_chiller[chiller_type]

        input_df_for_pred = pd.DataFrame([[current_total_load_watts, num_active_chillers_of_type]],
                                         columns=feature_cols_for_prediction)

        predictions = {}
        # models_for_type의 key는 'main_차압압력_bar' 등 일반화된 이름임
        for output_key, model in models_for_type.items():
            try:
                pred_val = model.predict(input_df_for_pred)[0]
                predictions[output_key] = pred_val
            except Exception as e:
                print(f"    오류: {output_key} 예측 중 오류 - {e}. 기본값 사용.")
                if "차압압력" in output_key: predictions[output_key] = 1.5
                elif "차압개도율" in output_key: predictions[output_key] = 60.0 # 메인/서브 동일 기본값
                else: predictions[output_key] = 0

        expected_output_keys = ["main_차압압력_bar", "main_차압개도율_percent", "sub_차압개도율_percent"]
        for key in expected_output_keys:
            if key not in predictions:
                print(f"    경고: 예측 결과에 {key}가 누락되어 기본값으로 채웁니다.")
                if "차압압력" in key: predictions[key] = 1.5
                else: predictions[key] = 60.0
        return predictions


class ChillerOptimizer:
    def __init__(self, num_low_temp_chillers=8, num_high_temp_chillers=8):
        self.num_low_temp_chillers = num_low_temp_chillers
        self.num_high_temp_chillers = num_high_temp_chillers
        self.chiller_specs = {
            "low_temp": [{"id": i, "max_capacity_watts": 200000, "cop_points": {0.3: 3.0, 0.55: 3.8, 0.8: 3.5, 1.0: 3.0}} for i in range(num_low_temp_chillers)],
            "high_temp": [{"id": i, "max_capacity_watts": 250000, "cop_points": {0.3: 3.2, 0.55: 4.0, 0.8: 3.7, 1.0: 3.2}} for i in range(num_high_temp_chillers)]
        }
        for type_key in self.chiller_specs:
            self.chiller_specs[type_key] = sorted(self.chiller_specs[type_key], key=lambda x: self._get_cop_at_load(x['cop_points'], 0.55), reverse=True)
        self.target_load_percentage = 0.55
        self.min_load_percentage = 0.30
        self.max_load_percentage = 0.90

    def _get_cop_at_load(self, cop_points, load_percentage):
        if not cop_points: return 3.0
        load_percentage = np.clip(load_percentage, 0.0, 1.0)
        if load_percentage == 0: return 0
        sorted_load_points = sorted(cop_points.keys())
        if load_percentage <= sorted_load_points[0]: return cop_points[sorted_load_points[0]]
        if load_percentage >= sorted_load_points[-1]: return cop_points[sorted_load_points[-1]]
        for i in range(len(sorted_load_points) - 1):
            x1, y1 = sorted_load_points[i], cop_points[sorted_load_points[i]]
            x2, y2 = sorted_load_points[i+1], cop_points[sorted_load_points[i+1]]
            if x1 <= load_percentage < x2:
                return y1 + (y2 - y1) * (load_percentage - x1) / (x2 - x1) if (x2-x1) != 0 else y1
        return cop_points[sorted_load_points[-1]]

    def optimize_chiller_operation(self, required_total_load_watts, chiller_type="low_temp"):
        specs_list = self.chiller_specs[chiller_type]
        num_total_chillers = len(specs_list)
        best_config = {'num_active': 0, 'active_chiller_ids': [], 'load_percentages': [], 'total_power': float('inf'), 'avg_load_percentage': 0, 'meets_target_load_rate': False}

        if required_total_load_watts <= 0: return 0, [], [], 0, 0, True

        for num_active in range(1, num_total_chillers + 1):
            current_chiller_selection_specs = specs_list[:num_active]
            total_max_cap_selected = sum(s['max_capacity_watts'] for s in current_chiller_selection_specs)
            if total_max_cap_selected * self.max_load_percentage < required_total_load_watts: continue
            if total_max_cap_selected * self.min_load_percentage > required_total_load_watts and required_total_load_watts > 0:
                if num_active > 1 : continue

            load_per_chiller_target = required_total_load_watts / num_active
            current_load_percentages, current_total_power, possible_combination, active_ids_temp = [], 0, True, []
            for chiller_spec in current_chiller_selection_specs:
                active_ids_temp.append(chiller_spec['id'])
                load_percentage_this_chiller = load_per_chiller_target / chiller_spec['max_capacity_watts']
                if not (self.min_load_percentage <= load_percentage_this_chiller <= self.max_load_percentage):
                    possible_combination = False; break
                current_load_percentages.append(load_percentage_this_chiller)
                cop = self._get_cop_at_load(chiller_spec['cop_points'], load_percentage_this_chiller)
                if cop <= 0: possible_combination = False; break
                current_total_power += (chiller_spec['max_capacity_watts'] * load_percentage_this_chiller) / cop

            if not possible_combination: continue
            avg_load_percentage = sum(current_load_percentages) / num_active if num_active > 0 else 0
            meets_target = abs(avg_load_percentage - self.target_load_percentage) < 0.05

            is_better = False
            if not best_config['active_chiller_ids']: is_better = True
            elif best_config['meets_target_load_rate'] and meets_target:
                if current_total_power < best_config['total_power']: is_better = True
            elif meets_target and not best_config['meets_target_load_rate']: is_better = True
            elif not best_config['meets_target_load_rate'] and not meets_target:
                if abs(avg_load_percentage - self.target_load_percentage) < abs(best_config['avg_load_percentage'] - self.target_load_percentage):
                    is_better = True
                elif abs(avg_load_percentage - self.target_load_percentage) == abs(best_config['avg_load_percentage'] - self.target_load_percentage):
                    if current_total_power < best_config['total_power']: is_better = True

            if is_better:
                best_config.update({'num_active': num_active, 'active_chiller_ids': sorted(active_ids_temp), 'load_percentages': current_load_percentages,
                                    'total_power': current_total_power, 'avg_load_percentage': avg_load_percentage, 'meets_target_load_rate': meets_target})

        if not best_config['active_chiller_ids'] and required_total_load_watts > 0 :
            num_active = num_total_chillers
            active_ids_temp = [s['id'] for s in specs_list]
            load_per_chiller = required_total_load_watts / num_active if num_active > 0 else 0
            current_load_percentages, current_total_power = [], 0
            for chiller_spec in specs_list:
                load_percentage = load_per_chiller / chiller_spec['max_capacity_watts'] if chiller_spec['max_capacity_watts'] > 0 else 0
                effective_load_perc_for_cop = np.clip(load_percentage, self.min_load_percentage, self.max_load_percentage)
                current_load_percentages.append(load_percentage)
                cop = self._get_cop_at_load(chiller_spec['cop_points'], effective_load_perc_for_cop)
                if cop <= 0: cop = 1.0
                current_total_power += (chiller_spec['max_capacity_watts'] * load_percentage) / cop

            avg_lp_fallback = sum(lp for lp in current_load_percentages if lp <=1.0) / num_active if num_active > 0 else 0
            best_config.update({'num_active': num_active, 'active_chiller_ids': sorted(active_ids_temp), 'load_percentages': current_load_percentages,
                                'total_power': current_total_power, 'avg_load_percentage': avg_lp_fallback,
                                'meets_target_load_rate': abs(avg_lp_fallback - self.target_load_percentage) < 0.10})

        return best_config['num_active'], best_config['active_chiller_ids'], best_config['load_percentages'], best_config['total_power'], best_config['avg_load_percentage'], best_config['meets_target_load_rate']

class CoolingTowerOptimizer:
    def __init__(self, num_towers=9):
        self.tower_specs = [{"id": i, "max_capacity_watts": 300000 + np.random.randint(-2,3)*10000, "fan_power_watts": 7000 + np.random.randint(-5,6)*100} for i in range(num_towers)]
        # self.tower_specs = [
        #     {"id": 0, "max_capacity_watts": 300000, "fan_power_watts": 7000}, {"id": 1, "max_capacity_watts": 300000, "fan_power_watts": 7000},
        #     {"id": 2, "max_capacity_watts": 350000, "fan_power_watts": 8000}, {"id": 3, "max_capacity_watts": 320000, "fan_power_watts": 7500},
        #     {"id": 4, "max_capacity_watts": 320000, "fan_power_watts": 7500}
        # ] + [{"id": i, "max_capacity_watts": 330000, "fan_power_watts": 7700} for i in range(5, num_towers)]
        self.num_towers = len(self.tower_specs)
        self.sorted_towers = sorted(self.tower_specs, key=lambda x: x['fan_power_watts'] / x['max_capacity_watts'] if x['max_capacity_watts'] > 0 else float('inf'))

    def optimize_tower_operation(self, total_heat_rejection_load_watts):
        active_tower_ids, current_total_capacity, current_total_fan_power = [], 0, 0
        if total_heat_rejection_load_watts <= 0: return [], 0, 0, 0
        for tower in self.sorted_towers:
            if current_total_capacity >= total_heat_rejection_load_watts: break
            active_tower_ids.append(tower['id']); current_total_capacity += tower['max_capacity_watts']; current_total_fan_power += tower['fan_power_watts']
        return sorted(active_tower_ids), current_total_fan_power, current_total_capacity, len(active_tower_ids)

class HVACSystemFacade:
    def __init__(self, oac_model_instance, chiller_model_instance):
        self.oac_model = oac_model_instance
        self.chiller_model = chiller_model_instance
        self.chiller_optimizer = ChillerOptimizer()
        self.cooling_tower_optimizer = CoolingTowerOptimizer()
        self.air_mass_flow_rate_per_oac_unit_kg_s = 12.0
        self.num_oac_units = 14

    def predict_and_optimize(self, outdoor_temp_c, outdoor_rh_percent, oac_set_points,
                             chilled_water_supply_set_temp, modified_oac_coil_states=None):
        oac_results_single_unit = self.oac_model.predict_coil_open_rates(
            outdoor_temp_c, outdoor_rh_percent, oac_set_points, modified_oac_coil_states
        )

        delta_h_precooling_j_kg = oac_results_single_unit.get('precooling', {}).get('delta_h_j_kg', 0)
        total_precooling_load_watts = abs(delta_h_precooling_j_kg * self.air_mass_flow_rate_per_oac_unit_kg_s * self.num_oac_units)

        delta_h_cooling_j_kg = oac_results_single_unit.get('cooling', {}).get('delta_h_j_kg', 0)
        total_cooling_load_watts = abs(delta_h_cooling_j_kg * self.air_mass_flow_rate_per_oac_unit_kg_s * self.num_oac_units)

        optim_low_ch_count, active_low_ids, load_dist_low, power_low, avg_load_low, meets_target_low = \
            self.chiller_optimizer.optimize_chiller_operation(total_cooling_load_watts, chiller_type="low_temp")
        chiller_preds_low_temp = {}
        if optim_low_ch_count > 0 :
            chiller_preds_low_temp = self.chiller_model.predict_pressure_valve(
                total_cooling_load_watts, optim_low_ch_count, chiller_type="low_temp"
            )

        optim_high_ch_count, active_high_ids, load_dist_high, power_high, avg_load_high, meets_target_high = \
            self.chiller_optimizer.optimize_chiller_operation(total_precooling_load_watts, chiller_type="high_temp")
        chiller_preds_high_temp = {}
        if optim_high_ch_count > 0 :
            chiller_preds_high_temp = self.chiller_model.predict_pressure_valve(
                total_precooling_load_watts, optim_high_ch_count, chiller_type="high_temp"
            )

        total_heat_rejection_watts = total_precooling_load_watts + total_cooling_load_watts + power_low + power_high
        active_tower_ids, tower_fan_power, tower_total_capacity, active_tower_count = \
            self.cooling_tower_optimizer.optimize_tower_operation(total_heat_rejection_watts)

        return {
            "oac_predictions_per_unit": oac_results_single_unit,
            "calculated_loads": {"total_precooling_load_watts": total_precooling_load_watts, "total_cooling_load_watts": total_cooling_load_watts},
            "low_temp_chiller_optimization": {"optimal_active_count": optim_low_ch_count, "active_chiller_ids": active_low_ids, "load_distribution_percentage_per_active_chiller": load_dist_low, "total_estimated_power_watts": power_low, "average_load_percentage_of_active_chillers": avg_load_low, "meets_target_load_rate": meets_target_low, "predictions": chiller_preds_low_temp},
            "high_temp_chiller_optimization": {"optimal_active_count": optim_high_ch_count, "active_chiller_ids": active_high_ids, "load_distribution_percentage_per_active_chiller": load_dist_high, "total_estimated_power_watts": power_high, "average_load_percentage_of_active_chillers": avg_load_high, "meets_target_load_rate": meets_target_high, "predictions": chiller_preds_high_temp},
            "cooling_tower_optimization": {"total_heat_rejection_load_watts": total_heat_rejection_watts, "active_tower_ids": active_tower_ids, "active_tower_count": active_tower_count, "total_fan_power_watts": tower_fan_power, "total_active_capacity_watts": tower_total_capacity}
        }

if __name__ == '__main__':
    print("--- 1. 상세 더미 데이터 생성 중 ---")
    dummy_df = generate_dummy_data(num_rows=500, num_oac=14, num_low_ch=8, num_high_ch=8, num_ct=9) # 행 수 줄여서 테스트 시간 단축
    # print("더미 데이터 생성 완료. 일부 컬럼 확인:")
    # print(dummy_df[['datetime', 'outdoor_temp_c', 'oac_0_cooling_coil_open_rate', 'total_cooling_load_watts', 'low_temp_chiller_0_supply_냉수온도_c']].head())


    print("\n--- 2. 모델 객체 생성 및 학습 ---")
    oac_model_inst = OACModel()
    oac_model_inst.train_coil_models(dummy_df.copy())

    chiller_model_inst = ChillerModel()
    chiller_model_inst.train_pressure_valve_models(dummy_df.copy())

    print("\n--- 3. HVAC 시스템 Facade 생성 (학습된 모델 사용) ---")
    hvac_system = HVACSystemFacade(oac_model_instance=oac_model_inst,
                                   chiller_model_instance=chiller_model_inst)
    print("Facade 객체 생성 완료.")

    print("\n" + "="*20 + " 학습된 모델 기반 하절기 시나리오 실행 " + "="*20)
    outdoor_temp_summer = 32.0
    outdoor_humidity_summer = 75.0
    oac_target_set_points_summer = {
        'preheating': 18.0, 'precooling': 20.0, 'cooling': 12.0,
        'reheating': 23.0, 'humidification_rate': 0.0
    }
    chilled_water_supply_temp_set_summer = 7.0

    print(f"\n[입력 조건]")
    print(f"  외기 온도: {outdoor_temp_summer}°C, 외기 상대 습도: {outdoor_humidity_summer}%")
    print(f"  OAC 코일별 목표 설정 (대표 OAC 기준):")
    for coil, sp_val in oac_target_set_points_summer.items():
        if coil != 'humidification_rate': print(f"    - {coil.capitalize()} 목표 온도: {sp_val}°C")
        else: print(f"    - Humidification 목표 개도율: {sp_val*100:.1f}%")
    print("-" * 50)

    summer_results = hvac_system.predict_and_optimize(
        outdoor_temp_summer, outdoor_humidity_summer,
        oac_target_set_points_summer, chilled_water_supply_temp_set_summer
    )

    print("\n[OAC 코일별 예측 결과 (단일 대표 유닛 기준) - 학습된 모델 사용]")
    oac_preds = summer_results["oac_predictions_per_unit"]
    for coil_name in oac_model_inst.coil_order:
        data = oac_preds.get(coil_name)
        if isinstance(data, dict):
            target_display = data.get('target_outlet_temp_c', 'N/A')
            if coil_name == 'humidification': target_display = f"Rate {oac_target_set_points_summer.get('humidification_rate',0.0)*100:.1f}%"
            print(f"  코일: {coil_name.capitalize()} (목표: {target_display})")
            print(f"    예측 개도율: {data.get('open_rate', 0.0):.3f} ({(data.get('open_rate', 0.0)*100):.1f}%)")
            # print(f"    입구: {data.get('inlet_temp_c'):.1f}°C, {data.get('inlet_rh_percent'):.1f}% ({data.get('inlet_h_j_kg'):.0f} J/kg)")
            # print(f"    출구: {data.get('outlet_temp_c'):.1f}°C, {data.get('outlet_rh_percent'):.1f}% ({data.get('outlet_h_j_kg'):.0f} J/kg)")
    print(f"  최종 토출 (대표 OAC): {oac_preds.get('final_outlet_temp_c'):.1f}°C, {oac_preds.get('final_outlet_rh_percent'):.1f}%")

    print("\n[계산된 총 시스템 부하 (OAC 14대 기준)]")
    loads = summer_results["calculated_loads"]
    print(f"  총 예냉 코일 부하 (고온 냉동기 담당): {loads['total_precooling_load_watts']:,.0f} Watts")
    print(f"  총 주 냉각 코일 부하 (저온 냉동기 담당): {loads['total_cooling_load_watts']:,.0f} Watts")

    print("\n[저온 냉동기 시스템 최적화 및 예측 - 학습된 모델 사용]")
    low_temp_ch_opt = summer_results["low_temp_chiller_optimization"]
    print(f"  최적 가동 대수: {low_temp_ch_opt['optimal_active_count']} 대 (IDs: {low_temp_ch_opt['active_chiller_ids']})")
    print(f"  가동 냉동기 평균 부하율: {low_temp_ch_opt['average_load_percentage_of_active_chillers']:.2%}")
    print(f"  예측된 시스템 차압/밸브: {low_temp_ch_opt['predictions']}")

    print("\n[고온 냉동기 시스템 최적화 및 예측 - 학습된 모델 사용]")
    high_temp_ch_opt = summer_results["high_temp_chiller_optimization"]
    print(f"  최적 가동 대수: {high_temp_ch_opt['optimal_active_count']} 대 (IDs: {high_temp_ch_opt['active_chiller_ids']})")
    print(f"  가동 냉동기 평균 부하율: {high_temp_ch_opt['average_load_percentage_of_active_chillers']:.2%}")
    print(f"  예측된 시스템 차압/밸브: {high_temp_ch_opt['predictions']}")

    print("\n[냉각탑 최적화 결과]")
    ct_opt = summer_results["cooling_tower_optimization"]
    print(f"  총 방열 부하: {ct_opt['total_heat_rejection_load_watts']:,.0f} Watts")
    print(f"  최적 가동 대수: {ct_opt['active_tower_count']} 대 (IDs: {ct_opt['active_tower_ids']})")
    print("="*50)

--- 1. 상세 더미 데이터 생성 중 ---
상세 더미 데이터 생성 시작: 500 행, OAC 14대, 저온냉동기 8대, 고온냉동기 8대, 냉각탑 9대
상세 더미 데이터 생성 완료. DataFrame Shape: (500, 434)

--- 2. 모델 객체 생성 및 학습 ---

--- OAC 코일 모델 학습 시작 (대표 OAC: oac_0 사용) ---
  preheating 코일 모델 학습 중...


<ipython-input-1-90e46a7d3c6d>:281: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = temp_df_train[current_features_specific_names].fillna(method='ffill').fillna(method='bfill')


    preheating 모델 학습 완료. Test MSE: 0.0015
  precooling 코일 모델 학습 중...


<ipython-input-1-90e46a7d3c6d>:281: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = temp_df_train[current_features_specific_names].fillna(method='ffill').fillna(method='bfill')


    precooling 모델 학습 완료. Test MSE: 0.0009
  cooling 코일 모델 학습 중...


...

    humidification 모델 학습 완료. Test MSE: 0.0002
--- OAC 코일 모델 학습 완료 ---

--- 냉동기 차압/밸브 모델 학습 시작 ---
  low_temp_chiller 시스템 모델 학습 중...


<ipython-input-1-90e46a7d3c6d>:447: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = df_train_data[current_feature_cols].fillna(method='ffill').fillna(method='bfill')


    main_차압압력_bar 모델 학습 완료. Test MSE: 0.0091
    main_차압개도율_percent 모델 학습 완료. Test MSE: 28.8172
    sub_차압개도율_percent 모델 학습 완료. Test MSE: 30.4726
  high_temp_chiller 시스템 모델 학습 중...


<ipython-input-1-90e46a7d3c6d>:447: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = df_train_data[current_feature_cols].fillna(method='ffill').fillna(method='bfill')


    main_차압압력_bar 모델 학습 완료. Test MSE: 0.0151
    main_차압개도율_percent 모델 학습 완료. Test MSE: 24.1925
    sub_차압개도율_percent 모델 학습 완료. Test MSE: 28.9953
--- 냉동기 차압/밸브 모델 학습 완료 ---

--- 3. HVAC 시스템 Facade 생성 (학습된 모델 사용) ---
Facade 객체 생성 완료.

==================== 학습된 모델 기반 하절기 시나리오 실행 ====================

[입력 조건]
  외기 온도: 32.0°C, 외기 상대 습도: 75.0%
  OAC 코일별 목표 설정 (대표 OAC 기준):
    - Preheating 목표 온도: 18.0°C
    - Precooling 목표 온도: 20.0°C
    - Cooling 목표 온도: 12.0°C
    - Reheating 목표 온도: 23.0°C
    - Humidification 목표 개도율: 0.0%
--------------------------------------------------

[OAC 코일별 예측 결과 (단일 대표 유닛 기준) - 학습된 모델 사용]
  코일: Preheating (목표: 18.0)
    예측 개도율: 0.002 (0.2%)
  코일: Precooling (목표: 20.0)
    예측 개도율: 0.957 (95.7%)
  코일: Cooling (목표: 12.0)
    예측 개도율: 0.616 (61.6%)
  코일: Reheating (목표: 23.0)
    예측 개도율: 0.977 (97.7%)
  코일: Humidification (목표: Rate 0.0%)
    예측 개도율: 0.000 (0.0%)
  최종 토출 (대표 OAC): 16.8°C, 50.2%

[계산된 총 시스템 부하 (OAC 14대 기준)]
  총 예냉 코일 부하 (고온 냉동기 담당): 3,214,727 Watts
  총 주 냉